In [9]:
import jax.numpy as jnp
from model import Model  # Assume the refactored code is saved in model.py
from rbm import RBM

In [1]:
from joblib import Parallel

In [11]:
def test_batch_jnp_nojit(N = 10000, weight = 0.1):
    model = Model(4,4)
    rbm = RBM(model)
    spin = model.get_random_spins()
    rbm.set_evaluate_function(rbm.evaluate_function_dummy(weight))
    rbm.evaluate(spin)
    rbm.metropolis_step(spin)
    batch = rbm.create_batch(N)
    test = 0
    for spins in batch:
        test += spins[0,0,0]
    mean = test / N
    expected_mean = (weight**2)/(weight ** 2 + 1)
    print(test/N, jnp.sqrt(expected_mean * (1-expected_mean)/N) * 4)
    assert jnp.abs(mean -  (weight**2)/(weight ** 2 + 1)) < jnp.sqrt(expected_mean * (1-expected_mean)/N) * 4, f"mean = {mean}, expected = {1 * (weight**2)/(weight ** 2 + 1)}"

In [12]:
test_batch_jnp_nojit(N = 10000)

0.0088 0.003960396
